## Assignment problem
Assignment problem example with 6 teaching assistants needgin to be assigned to 3 projects

In [1]:
import numpy as np
import pandas as pd
import os
import networkx as nx
import random
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

from pyomo.environ import *

In [2]:
############
### Sets ###
############

# Projects
# Note: the first 3 are the "real" projects. Projects
# 4, 5, and 6 are a replica of 1, 2, and 3 respectively
P = {1:{'Name':'Project 1',1:2,2:1,3:4,4:3,5:7,6:5},
     2:{'Name':'Project 2',1:1,2:4,3:2,4:1,5:4,6:9},
     3:{'Name':'Project 3',1:3,2:2,3:5,4:3,5:6,6:4},
     4:{'Name':'Project 4',1:2,2:1,3:4,4:3,5:7,6:5},
     5:{'Name':'Project 5',1:1,2:4,3:2,4:1,5:4,6:9},
     6:{'Name':'Project 6',1:3,2:2,3:5,4:3,5:6,6:4}}

# Teaching assistants
T = {1:{'Name':'TA 1'},
     2:{'Name':'TA 2'},
     3:{'Name':'TA 3'},
     4:{'Name':'TA 4'},
     5:{'Name':'TA 5'},
     6:{'Name':'TA 6'}}

In [3]:
model = ConcreteModel()

# Define sets
model.Projects = Set(initialize=P.keys())
model.TAs      = Set(initialize=T.keys())

# Define parameters
model.C_pt = Param(model.Projects,model.TAs, initialize={(p,t):v[t] 
       for p,v in P.items() for t in T.keys()}, within=Any)

# Define decision variables
model.x = Var(model.Projects,model.TAs, within=Binary)

# Define objective function
model.obj = Objective(expr=sum(model.C_pt[p,t]*model.x[p,t] 
                               for p in model.Projects
                               for t in model.TAs), sense=minimize)

# Define constraints
model.project_assigned_once = ConstraintList()
for p in model.Projects:
    model.project_assigned_once.add(expr=sum(model.x[p,t] 
                                         for t in model.TAs)==1)
    
model.TA_assigned_once = ConstraintList()
for t in model.TAs:
    model.project_assigned_once.add(expr=sum(model.x[p,t] 
                                         for p in model.Projects)==1) 
    

In [ ]:
# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model) 

In [9]:
# Print the results
print('Overall assignment cost:', model.obj())
for t in model.TAs:
    for p in model.Projects:
        print(f'TA {t} - Project {p}: {model.x[(p,t)].value}')  


Overall assignment cost: 16.0
TA 1 - Project 1: 1.0
TA 1 - Project 2: -0.0
TA 1 - Project 3: 0.0
TA 1 - Project 4: -0.0
TA 1 - Project 5: -0.0
TA 1 - Project 6: -0.0
TA 2 - Project 1: 0.0
TA 2 - Project 2: -0.0
TA 2 - Project 3: -0.0
TA 2 - Project 4: 1.0
TA 2 - Project 5: -0.0
TA 2 - Project 6: 0.0
TA 3 - Project 1: -0.0
TA 3 - Project 2: -0.0
TA 3 - Project 3: -0.0
TA 3 - Project 4: -0.0
TA 3 - Project 5: 1.0
TA 3 - Project 6: -0.0
TA 4 - Project 1: -0.0
TA 4 - Project 2: 0.0
TA 4 - Project 3: -0.0
TA 4 - Project 4: -0.0
TA 4 - Project 5: 0.0
TA 4 - Project 6: 1.0
TA 5 - Project 1: -0.0
TA 5 - Project 2: 1.0
TA 5 - Project 3: -0.0
TA 5 - Project 4: -0.0
TA 5 - Project 5: -0.0
TA 5 - Project 6: -0.0
TA 6 - Project 1: -0.0
TA 6 - Project 2: -0.0
TA 6 - Project 3: 1.0
TA 6 - Project 4: -0.0
TA 6 - Project 5: -0.0
TA 6 - Project 6: -0.0
